# **Arxiv metadata Analytics with PySpark DF: JSON case study**

### Udemy Course: Best Hands-on Big Data Practices and Use Cases using PySpark

### Author: Amin Karami (PhD, FHEA)
#### email: amin.karami@ymail.com

In [ ]:
########## ONLY in Colab ##########
#!pip3 install pyspark
########## ONLY in Colab ##########

In [1]:
########## ONLY in Ubuntu Machine ##########
# Load Spark engine
#!pip3 install -q findspark
import findspark
findspark.init()
########## ONLY in Ubuntu Machine ##########

In [3]:
# import SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [4]:
# Read and Load Data to Spark
df = spark.read.json("json_data/arxiv-metadata-oai-snapshot.json")
print(df.printSchema())

root
 |-- abstract: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- authors_parsed: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- categories: string (nullable = true)
 |-- comments: string (nullable = true)
 |-- doi: string (nullable = true)
 |-- id: string (nullable = true)
 |-- journal-ref: string (nullable = true)
 |-- license: string (nullable = true)
 |-- report-no: string (nullable = true)
 |-- submitter: string (nullable = true)
 |-- title: string (nullable = true)
 |-- update_date: string (nullable = true)
 |-- versions: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- created: string (nullable = true)
 |    |    |-- version: string (nullable = true)

None


In [5]:
# check the partitions
print(df.rdd.getNumPartitions())
#you can change num of partitions by repartition method.

25


## Question 1: Create a new Schema

In [6]:
#Define a schema
from pyspark.sql.types import *
Schema = StructType([
    StructField('authors',StringType(),True),
    StructField('categories',StringType(),True),
    StructField('license',StringType(),True),
    StructField('comments',StringType(),True),
    StructField('abstract',StringType(),True),
    StructField('versions',ArrayType(StringType()),True)
])
print(Schema)

StructType([StructField('authors', StringType(), True), StructField('categories', StringType(), True), StructField('license', StringType(), True), StructField('comments', StringType(), True), StructField('abstract', StringType(), True), StructField('versions', ArrayType(StringType(), True), True)])


## Question 2: Binding Data to a Schema

In [10]:
df = spark.read.json("json_data/arxiv-metadata-oai-snapshot.json", schema = Schema)
print(df.show())

+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+
|             authors|       categories|             license|            comments|            abstract|            versions|
+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+
|C. Bal\'azs, E. L...|           hep-ph|                null|37 pages, 15 figu...|  A fully differe...|[{"version":"v1",...|
|Ileana Streinu an...|    math.CO cs.CG|http://arxiv.org/...|To appear in Grap...|  We describe a n...|[{"version":"v1",...|
|         Hongjun Pan|   physics.gen-ph|                null| 23 pages, 3 figures|  The evolution o...|[{"version":"v1",...|
|        David Callan|          math.CO|                null|            11 pages|  We show that a ...|[{"version":"v1",...|
|Wael Abu-Shammala...|  math.CA math.FA|                null|                null|  In this paper w...|[{"version":"v1",...|


## Question 3: Missing values for "comments" and "license" attributes

In [11]:
#drop
df = df.dropna(subset=["comments"])
#replace
df = df.fillna(value="unknown", subset=["license"])
print(df.show())

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|             authors|          categories|             license|            comments|            abstract|            versions|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|C. Bal\'azs, E. L...|              hep-ph|             unknown|37 pages, 15 figu...|  A fully differe...|[{"version":"v1",...|
|Ileana Streinu an...|       math.CO cs.CG|http://arxiv.org/...|To appear in Grap...|  We describe a n...|[{"version":"v1",...|
|         Hongjun Pan|      physics.gen-ph|             unknown| 23 pages, 3 figures|  The evolution o...|[{"version":"v1",...|
|        David Callan|             math.CO|             unknown|            11 pages|  We show that a ...|[{"version":"v1",...|
|Y. H. Pong and C....|   cond-mat.mes-hall|             unknown|6 pages, 4 figure...|  We study the tw..

## Question 4: Get the author names who published a paper in a 'math' category

In [13]:
#register DF.because we are going to use SparkSql
df.createOrReplaceTempView("Archive")
sql_query = """
select authors from archive
where categories like 'math%'
"""
print(spark.sql(sql_query).show())
print(spark.sql(sql_query).count())

+--------------------+
|             authors|
+--------------------+
|Ileana Streinu an...|
|        David Callan|
|  Sergei Ovchinnikov|
|Clifton Cunningha...|
|        Koichi Fujii|
|         Norio Konno|
|Simon J.A. Malham...|
|Robert P. C. de M...|
|  P\'eter E. Frenkel|
|          Mihai Popa|
|   Debashish Goswami|
|      Mikkel {\O}bro|
|Nabil L. Youssef,...|
|         Boris Rubin|
|         A. I. Molev|
| Branko J. Malesevic|
|   John W. Robertson|
|     Yu.N. Kosovtsov|
|        Osamu Fujino|
|Stephen C. Power ...|
+--------------------+
only showing top 20 rows

None
304590


## Question 5: Get linceses with 5 or more letters in the "abstract"

In [17]:
sql_query = """
select distinct(license) from Archive
where abstract REGEXP '%\(([A-Za-z][^_ /\\<>]{5,})\)%'
"""
print(spark.sql(sql_query).show())

+--------------------+
|             license|
+--------------------+
|http://arxiv.org/...|
|http://creativeco...|
|http://creativeco...|
|http://creativeco...|
|             unknown|
+--------------------+



## Question 6: Extract the statistic of the number of pages for unknown licenses

In [21]:
import re
def get_Page(line):
    search = re.findall('\d+ pages' , line)
    if search:
        return int(search[0].split(" ")[0])
    else:
        return 0
spark.udf.register("PageNumbers",get_Page)
sql_query="""
select avg(PageNumbers(comments)) as avg,sum(PageNumbers(comments)) as sum,std(PageNumbers(comments)) as std from Archive
where license = 'unknown'
"""
spark.sql(sql_query).show()

+------------------+---------+------------------+
|               avg|      sum|               std|
+------------------+---------+------------------+
|13.368011068572079|5642584.0|16.777518213632323|
+------------------+---------+------------------+

